In [31]:
import sys
!{sys.executable} -m pip install tweepy
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install -U textblob
!{sys.executable} -m pip install preprocessor
!{sys.executable} -m pip install tweet-preprocessor
!{sys.executable} -m pip install sklearn
!{sys.executable} -m pip install argparse
!{sys.executable} -m pip install -U nltk



Requirement already up-to-date: textblob in /anaconda3/lib/python3.7/site-packages (0.15.3)
Requirement already up-to-date: nltk in /anaconda3/lib/python3.7/site-packages (3.4.1)


In [32]:
import tweepy
import os
import sklearn
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json
import pandas as pd
import csv
import re #regular expression
from textblob import TextBlob
import string
import preprocessor.api as p
import argparse
import nltk
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords

In [33]:
#Twitter credentials for the app
consumer_key = 'Lydypy5GRHslhuWsXTAagVFpO'
consumer_secret = 'K9HA6MyfRWm73G50WHvzBPxfY0gWfJRk5ajcUmGRCg4e9NiM69'
access_key= '789687511-BGbhUzj8zVLk9HeKKxrCZnzJ21xb3qXqZMHyf0gX'
access_secret = 'kIDmi6vhOiePyEIZ5XXrOV8rl0xLOe5wLQ2XbhH2qCLsr'

In [34]:
#pass twitter credentials to tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

In [35]:
#declare file paths as follows for three files
democrat_tweets = "/Users/sunnyli/Documents/Spring 2019/CS 221/221-Project/data/house_democrat_data.csv"
republican_tweets = "/Users/sunnyli/Documents/Spring 2019/CS 221/221-Project/data/house_republican_data.csv"

In [36]:
#columns of the csv file
COLS = ['id','original_text', 'clean_text','polarity', 'subjectivity', 'hashtags']

In [37]:
#HappyEmoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

In [38]:
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

In [39]:
#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)


In [40]:
#combine sad and happy emoticons
emoticons = emoticons_happy.union(emoticons_sad)

In [41]:
def clean_tweets(tweet):
 
    #stop_words = set(stopwords.words('english'))
#after tweepy preprocessing the colon symbol left remain after      #removing mentions
    tweet = re.sub(r'‚Ä¶', '', tweet)
    tweet = re.sub(r'RT *[^:]*:','', tweet);
    tweet = re.sub(r'\.@[a-zA-Z]*:','', tweet);
    tweet = re.sub(r':', '', tweet)
#remove @ symbol
    tweet = tweet.replace(".@", "")
    tweet = tweet.replace("@", "")
#replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
#remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)
#remove url
    tweet = re.sub(r"http\S+", "", tweet)
#filter using NLTK library append it to a string
    filtered_tweet = []
    word_tokens = word_tokenize(tweet)
#looping through conditions
    for w in word_tokens:
#check tokens against stop words , emoticons and punctuations
        if w not in emoticons and w not in string.punctuation:
            if w[0] == ".":
                continue
            w = re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ', w)
            w = w.lower()
            
            if len(w) == 1 and w not in ['a', 'e', 'i','o','u']:
                continue
            elif len(w) == 2 and w[0] in string.punctuation:
                continue
            filtered_tweet.append(w.lower())
    return ' '.join(filtered_tweet)
    #print(word_tokens)
    #print(filtered_sentence)return tweet

In [42]:
contractions_dict = { 
"ain't": "have not",
"aren't": "are not",
"can't": "cannot",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'll": "he will",
"he's": "he has",
"how'd": "how did",
"how'll": "how will",
"how's": "how are",
"i'd": "I would",
"i'll": "I will",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"o'clock": "of the clock",
"oughtn't": "ought not",
"she'd": "she would",
"she'll": "she will",
"she's": "she has",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there would",
"there's": "there has",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"y'all": "you all",
"you'd": "you would",
"you'll": "you will",
"you're": "you are",
"you've": "you have"
}


In [43]:
contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()), flags = re.IGNORECASE)
def expand_contractions(s, contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0).lower()]
    return contractions_re.sub(replace, s)

In [44]:
def write_tweets(handle, file):
    #If the file exists, then read the existing data from the CSV file.
    #if os.path.exists(file):
        #df = pd.read_csv(file, header=0)
   # else:
        #df = pd.DataFrame(columns=COLS)
    #page attribute in tweepy.cursor and iteration
    for page in tweepy.Cursor(api.user_timeline, id=handle,
                              count=200, include_rts=True, tweet_mode = 'extended').pages():
        for status in page:
            new_entry = []
            status = status._json
    
            if "retweeted_status" in status:
                status['full_text'] = status["retweeted_status"]['full_text']
            
            original_tweet = status['full_text']
            
            status['full_text'] = status['full_text'].replace("&amp;", "and")
            status['full_text'] = status['full_text'].replace("#", "")
            status['full_text'] = status['full_text'].replace("’", "'")
            status['full_text'] = expand_contractions(status['full_text'])
            
            status['full_text'] = status['full_text'].replace(".@", "")
            status['full_text'] = status['full_text'].replace("@", "")
            clean_text = p.clean(status['full_text'])
            filtered_tweet=clean_tweets(clean_text)

            blob = TextBlob(filtered_tweet)
            Sentiment = blob.sentiment     
            polarity = Sentiment.polarity
            subjectivity = Sentiment.subjectivity
            
            new_entry += [status['id'],filtered_tweet, original_tweet, polarity,subjectivity]
            hashtags = ", ".join([hashtag_item['text'] for hashtag_item in status['entities']['hashtags']])
            new_entry.append(hashtags) #append the hashtags
        
            with open(file, "a+", newline='') as fp:
                wr = csv.writer(fp, dialect='excel')
                wr.writerow(new_entry)
    
            #single_tweet_df = pd.DataFrame([new_entry], columns=COLS)
            #df_final = df.append(single_tweet_df, ignore_index=True)
            
            #csvFile = open(file, 'a+' ,encoding='utf-8')
            #df.to_csv(csvFile, mode='a', columns=COLS, index=False, encoding="utf-8")

In [45]:
democrat_handle = "HouseDemocrats"
republican_handle = "HouseGOP"

write_tweets(democrat_handle, democrat_tweets)
write_tweets(republican_handle, republican_tweets)

print ('finished')

finished
